### Project Setup 

In [1]:
from matplotlib import figure
from matplotlib.markers import MarkerStyle
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import smtplib
import pyodbc
import sqlalchemy 
from sqlalchemy import create_engine

### Database Connection

In [2]:
conn = pyodbc.connect('DRIVER={SQL Server};'
                          'SERVER=KANWKS4032;'
                          'DATABASE=OperatorPerformance;'
                          'UID=sa;'
                          'PWD=test1234')

### 1. Read all data

In [3]:
all_query = (
    '''
                SELECT [ShiftDate]
                    ,[MineNo]
                    ,[OperatorName]
                    ,[Digger]
                    ,[Model]
                    ,[Cutback]
                    ,[Loading]
                    ,[Spotting]
                FROM [OperatorPerformance].[dbo].[DiggerOperatorPerformance]
                ORDER BY [Loading] DESC
            '''
)

In [4]:
df = pd.read_sql(all_query, conn)

C:\Users\mnzunda\AppData\Local\Temp\ipykernel_35796\322622272.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(all_query, conn)


In [5]:
df

,ShiftDate,MineNo,OperatorName,Digger,Model,Cutback,Loading,Spotting
0,2022-11-27,001146,Oscar Ngulube,EX58,LIEBHERR 9350 E,M17,18171.0,0.0
1,2022-11-27,004250,Mukuka Mulenga,EX69,LIEBHERR 9250,M12,18040.0,0.0
2,2022-11-27,008629,Leonard Kalukangu,EX108,LIEBHERR 9350,M11,17924.0,56.0
3,2022-11-27,004471,Terence Chilufya,EX59,LIEBHERR 9350 E,M13,17905.0,0.0
4,2022-03-09,008547,Geoffery Kapini,EX72,LIEBHERR 9350,M17,5454.0,0.0
...,...,...,...,...,...,...,...,...
757361,2022-10-05,004162,Kenneth Kasauntu,EX62,LIEBHERR 984,M11,NaN,NaN
757362,2022-10-05,008542,Brighton Malambo,EX70,LIEBHERR 9250,M12,NaN,NaN
757363,2022-11-03,008600,Jason Chumfwa,EX72,LIEBHERR 9350,M12,NaN,NaN
757364,2022-11-02,003570,Titus Kwenda,EX59,LIEBHERR 9350 E,M13,NaN,NaN


### 2. Transactions Per Operator
* 2.1 Find count of transactions per operator
* 2.2 Find avearage count of transactions for all operators
* 2.3 Find count of transactions per operator above average

#### 2.1 Transaction count per operator

In [ ]:
operator_count_query = (
        '''
                    SELECT COUNT(*) AS LoadCount, [MineNo]
                    FROM [OperatorPerformance].[dbo].[DiggerOperatorPerformance]
                    GROUP BY [MineNo]
                    ORDER BY COUNT(*) DESC
                '''
    )

In [ ]:
df_operator_count = pd.read_sql(operator_count_query, conn)

In [ ]:
df_operator_count 

##### Export result to csv file

In [ ]:
df_operator_count.to_csv('operator_count.csv')

##### Barchart - Transactions per operator

In [ ]:
df_operator_count.LoadCount.plot.barh()

##### LineChart - Transactions per operator

In [ ]:
df_operator_count['LoadCount'].plot()

#### 2.2 Average Transactions Per Operator

In [ ]:
avg_operator_transactions = (
        '''
                    SELECT AVG(Transactions.LoadCount) AS AverageTransactions
                    FROM 
                    (SELECT COUNT(*) AS LoadCount, [MineNo]
                    FROM [OperatorPerformance].[dbo].[DiggerOperatorPerformance]
                    GROUP BY [MineNo]) as Transactions
                '''
    )

In [ ]:
df_avg_operator_transactions = pd.read_sql(avg_operator_transactions, conn)
df_avg_operator_transactions

#### 2.3 Transactions above average per operator

In [ ]:
top_operator_count_query = (
        '''
                    SELECT TOP 79 COUNT(*) AS LoadCount, [MineNo]
                    FROM [OperatorPerformance].[dbo].[DiggerOperatorPerformance]
                    GROUP BY [MineNo]
                    ORDER BY COUNT(*) DESC
                '''
    )

In [ ]:
df_top_operator_count = pd.read_sql(top_operator_count_query, conn)

In [ ]:
df_top_operator_count 

In [ ]:
df_top_operator_count.LoadCount.plot.barh()

In [ ]:
df_top_operator_count['LoadCount'].plot()

### 3. Number of Transactions 
* 3.1 Cutback
* 3.2 Digger
* 3.2 Model

#### 3.1 Transactions per Cutback

In [ ]:
count_cutback_query = (
        '''
                    SELECT COUNT(*) AS CutbackTrans, [Cutback]
                    FROM [OperatorPerformance].[dbo].[DiggerOperatorPerformance]
                    GROUP BY [Cutback]
                    ORDER BY COUNT(*) DESC
                '''
    )

In [ ]:
df_count_cutback = pd.read_sql(count_cutback_query, conn)

In [ ]:
df_count_cutback

#### 3.2 Transactions per Digger

In [ ]:
count_digger_query = (
        '''
                    SELECT COUNT(*) AS DiggerTrans, [Digger]
                    FROM [OperatorPerformance].[dbo].[DiggerOperatorPerformance]
                    GROUP BY [Digger]
                    ORDER BY COUNT(*) DESC
                '''
    )

In [ ]:
df_count_digger = pd.read_sql(count_digger_query, conn)

In [ ]:
df_count_digger

#### 3.3 Transactions per Model

In [ ]:
count_model_query = (
        '''
                    SELECT COUNT(*) AS CutbackTrans, [Model]
                    FROM [OperatorPerformance].[dbo].[DiggerOperatorPerformance]
                    GROUP BY [Model]
                    ORDER BY COUNT(*) DESC
                '''
    )

In [ ]:
df_count_model = pd.read_sql(count_model_query, conn)

In [ ]:
df_count_model

### 4. Mean and Standard Deviation for Loading and Spotting Time
* 4.1 Standard Deviation
* 4.2 Mean

#### 4.1 Standard Deviation for Loading time and Spotting time

In [18]:
df.std(axis = 0, skipna = True)

C:\Users\mnzunda\AppData\Local\Temp\ipykernel_35796\2996963263.py:1: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.std(axis = 0, skipna = True)


ShiftDate    102 days 17:02:25.301893337
Loading                       104.047509
Spotting                       69.503963
dtype: object

#### 4.2 Mean for Loading time and Spotting time

In [12]:
avg_loading_spotting_query = (
                '''
                    SELECT AVG([Loading]) AS AverageLoadingTime
                    FROM [OperatorPerformance].[dbo].[DiggerOperatorPerformance]
                '''
    )

In [13]:
df_avg_loading_spotting = pd.read_sql(avg_loading_spotting_query, conn)

C:\Users\mnzunda\AppData\Local\Temp\ipykernel_35796\867220978.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_avg_loading_spotting = pd.read_sql(avg_loading_spotting_query, conn)


In [14]:
df_avg_loading_spotting

,AverageLoadingTime
0,234.536949


In [15]:
avg_spotting_query = (
                '''
                    SELECT AVG([Spotting]) AS AverageSpottingTime
                    FROM [OperatorPerformance].[dbo].[DiggerOperatorPerformance]
                '''
    )

In [16]:
df_avg_spotting = pd.read_sql(avg_spotting_query, conn)

C:\Users\mnzunda\AppData\Local\Temp\ipykernel_35796\4049302563.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_avg_spotting = pd.read_sql(avg_spotting_query, conn)


In [17]:
df_avg_spotting

,AverageSpottingTime
0,47.696796
